In [6]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mpich is already the newest version (4.0-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [7]:
%%writefile hello_mpi.cpp
#include <mpi.h>
#include <iostream>
using namespace std;

int main(int argc, char** argv) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    if (size != 3) {
        if (rank == 0)
            cout << "This program must be run with exactly 3 processes.\n";
        MPI_Finalize();
        return 0;
    }

    int ARRAY_SIZE;
    int* full_array = nullptr;
    int* sub_array = nullptr;

    if (rank == 0) {
        cout << "Enter the length (must be divisible by 2): ";
        cin >> ARRAY_SIZE;
        if (ARRAY_SIZE % 2 != 0) {
            cout << "Array size must be divisible by 2.\n";
            MPI_Abort(MPI_COMM_WORLD, 1);
        }
        full_array = new int[ARRAY_SIZE];
        cout << "Enter elements of array:\n";
        for (int i = 0; i < ARRAY_SIZE; i++) {
            cin >> full_array[i];
        }
    }

    // Broadcast the array size to all processes
    MPI_Bcast(&ARRAY_SIZE, 1, MPI_INT, 0, MPI_COMM_WORLD);

    int half_size = ARRAY_SIZE / 2;
    sub_array = new int[half_size];

    // Scatter the array to rank 0 and 1
    MPI_Scatter(full_array, half_size, MPI_INT,
        sub_array, half_size, MPI_INT,
        0, MPI_COMM_WORLD);

    int partial_sum = 0;
    for (int i = 0; i < half_size; i++) {
        partial_sum += sub_array[i];
    }

    if (rank == 0) {
        double start_time = MPI_Wtime();
        MPI_Send(&partial_sum, 1, MPI_INT, 2, 0, MPI_COMM_WORLD);
        MPI_Send(&start_time, 1, MPI_DOUBLE, 2, 1, MPI_COMM_WORLD);
    }
    else if (rank == 1) {
        MPI_Send(&partial_sum, 1, MPI_INT, 2, 2, MPI_COMM_WORLD);
    }
    else if (rank == 2) {
        int sum1, sum2;
        double start_time, end_time;
        MPI_Recv(&sum1, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        MPI_Recv(&start_time, 1, MPI_DOUBLE, 0, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        MPI_Recv(&sum2, 1, MPI_INT, 1, 2, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        int final_sum = sum1 + sum2;
        end_time = MPI_Wtime();
        cout << "Final sum: " << final_sum << endl;
        cout << "Processing time: " << end_time - start_time << " seconds" << endl;
    }

    if (full_array) delete[] full_array;
    delete[] sub_array;

    MPI_Finalize();
    return 0;
}

Overwriting hello_mpi.cpp


In [9]:
!mpic++ hello_mpi.cpp -o hello_mpi


In [16]:
!echo -e "6\n1 2 3 4 5 6" > input.txt
#colab doesnt take input, that is why we make an input file which has array of length whatever we need(here anything divisble by 2) and feed it while executing.
#this cell is for only colab
#if using vs then dont use this line of code

In [15]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --oversubscribe -np 3 ./hello_mpi <input.txt


Enter the length (must be divisible by 2): Enter elements of array:
Final sum: 21
Processing time: 0 seconds
